In [1]:
import time
import csv

In [2]:
def generiranje_tock_kvadrat(n):
    # funkcija naključno generira 2n točk v enotskem kvadratu

    V = RDF^2 # vektorski prostor R^2
    tocke = [V.random_element(min=0, max=1) for _ in range(2*n)]
    return tocke

In [3]:
def generiranje_tock_krog(n):
    # funkcija naključno generira 2n točk v enotskem krogu

    V = RDF^2 # vektorski prostor R^2
    tocke = []
    while len(tocke) < 2*n:
        x = V.random_element(min=-1, max=1)
        if x.norm() <= 1:
            tocke.append(x)
    return tocke

In [4]:
def generiranje_tock_enakostranicni_trikotnik(n):
    # funkcija naključno generira 2n točk v enakostraničnem trikotniku

    V = RDF^2 # vektorski prostor R^2
    A = Matrix([[1, 1/2], [0, sqrt(3)/2]]) # linearna preslikava
    b = V([1/2, sqrt(3)/6]) # premik, da bo središče trikotnika v izhodišču
    ee = V([1, 1]) # vektor za preslikovanje pod x+y = 1

    tocke = [A*(v if sum(v) < 1 else ee-v) - b for _ in range(2*n) for v in [V.random_element(min=0, max=1)]]
    return tocke

In [5]:
def graf(n, generiranje_tock, norma=2):
    # funkcija iz naključno generiranih točk ustvari poln graf

    tocke = generiranje_tock(n)
    G = graphs.CompleteGraph(len(tocke))

    for u, v in G.edges(labels=False, sort = False):
        G.set_edge_label(u, v, (tocke[u] - tocke[v]).norm(norma)) # uteži na povezavah so odvisne od razdalj med točkami
    return G

In [6]:
def dvobarven_graf(n, generiranje_tock, norma=2):
    # funkcija iz naključno generiranih točk ustvari poln dvodelni graf

    tocke = generiranje_tock(n)
    G = graphs.CompleteBipartiteGraph(n, n)

    for u, v in G.edges(labels=False, sort = False):
        G.set_edge_label(u, v, (tocke[u] - tocke[v]).norm(norma)) # uteži na povezavah so odvisne od razdalj med točkami

    return G

In [7]:
def graf_izris(G):
    # funkcija nariše graf G

    H = Graph([(*e, N(w, digits=2)) for *e, w in G.edges(labels=True, sort = False)])
    H.set_pos(G.get_pos())

    return H.plot(edge_labels=True)

In [8]:
def clp(G):
    # funkcija izpiše pare vozlišč med katerimi so povezave v najcenejšem prirejanju M, vsoto cen povezav in nariše graf

    p = MixedIntegerLinearProgram(maximization=False)
    b = p.new_variable(binary=True)
    p.set_objective(sum([w * b[Set(e)] for *e, w in G.edges(labels=True, sort = False)])) # upoštevamo uteži povezav

    for v in G: # tukaj nas uteži ne zanimajo
        p.add_constraint(sum([b[Set(e)] for e in G.edges_incident(v, labels=False)]) == 1)

    cena = p.solve() # vrne vrednost ciljne funkcije
    b = p.get_values(b)

    M = [tuple(e) for e, i in b.items() if i]
    print(M)

    # vrne cene povezav v M
    x = [w for *e, w in G.edges(sort = False) if tuple(e) in M] # seznam cen povezav v M
    print(sum(x))

    # izrisovanje
    H = Graph([(*e, N(w, digits=2)) for *e, w in G.edges(labels=True, sort = False)])
    H.set_pos(G.get_pos())

    return H.plot(edge_colors={"red": M}, edge_labels=True) # graf H z rdeče pobarvanimi povezavami iz prirejanja

In [9]:
def clp_vsota_in_cas(G):
    # funkcija reši clp in vrne minimalno vsoto ter čas ki ga porabi za reševanje clp
    p = MixedIntegerLinearProgram(maximization=False)
    b = p.new_variable(binary=True)
    p.set_objective(sum([w * b[Set(e)] for *e, w in G.edges(labels=True, sort = False)])) # upoštevamo uteži povezav

    start = time.time()
    for v in G: # tukaj nas uteži ne zanimajo
        p.add_constraint(sum([b[Set(e)] for e in G.edges_incident(v, labels=False)]) == 1)

    cena = p.solve() # vrne vrednost ciljne funkcije
    end = time.time()

    b = p.get_values(b)

    M = [tuple(e) for e, i in b.items() if i]

    # vrne cene povezav v M
    x = [w for *e, w in G.edges(sort = False) if tuple(e) in M] # seznam cen povezav v M

    return sum(x), end-start

In [21]:
def vec_ponovitev(n, lik, stevilo_ponovitev, graf=graf, norma=2):
    vsota = []
    cas = []
    for i in range(stevilo_ponovitev):
        x = clp_vsota_in_cas(graf(n, lik, norma))
        vsota.append(x[0])
        cas.append(x[1])
    return vsota, cas

In [28]:
def zapis_podatkov(m, n, lik, stevilo_ponovitev, naslov_vsota, naslov_cas=None, graf=graf, norma=2):
    podatki_vsota = []
    podatki_cas = []
    for i in range(m, n+1):
        x = vec_ponovitev(i, lik, stevilo_ponovitev, graf, norma)
        podatki_vsota.append(x[0])
        podatki_cas.append(x[1])

    C = podatki_vsota
    with open(naslov_vsota, 'w') as f:
        c = csv.writer(f)
        c.writerows(C)
    if naslov_cas != None:
        C = podatki_cas
        with open(naslov_cas, 'w') as f:
            c = csv.writer(f)
            c.writerows(C)

In [33]:
# minimalne vsote in časovna zahtevnost najcenejšega prirejanja za različne n v navadnem polnem grafu in dvodelnem polnem grafu, 10 ponovitev
zapis_podatkov(1, 40, generiranje_tock_kvadrat, 10, 'n_od_1_do_40_vsota', 'n_od_1_do_40_cas')
zapis_podatkov(1, 40, generiranje_tock_kvadrat, 10, 'n_od_1_do_40_vsota_dvodelni_graf','n_od_1_do_40_cas_dvodelni_graf', dvobarven_graf)

In [29]:
# primerjava po območjih izbire točk: enotski kvadrat, enotski krog, enakostranični trikotnik
zapis_podatkov(1, 10, generiranje_tock_kvadrat, 50, 'n_od_1_do_10_primerjava_liki_kvadrat')
zapis_podatkov(1, 10, generiranje_tock_krog, 50, 'n_od_1_do_10_primerjava_liki_krog')
zapis_podatkov(1, 10, generiranje_tock_enakostranicni_trikotnik, 50, 'n_od_1_do_10_primerjava_liki_enakostranicni_trikotnik')

In [30]:
# primerjava po območjih izbire točk dvodelni graf
zapis_podatkov(1, 10, generiranje_tock_kvadrat, 50, 'n_od_1_do_10_primerjava_liki_dvodelni_graf_kvadrat', None, dvobarven_graf)
zapis_podatkov(1, 10, generiranje_tock_krog, 50, 'n_od_1_do_10_primerjava_liki_dvodelni_graf_krog', None, dvobarven_graf)
zapis_podatkov(1, 10, generiranje_tock_enakostranicni_trikotnik, 50, 'n_od_1_do_10_primerjava_liki_dvodelni_graf_enakostranicni_trikotnik', None, dvobarven_graf)

In [31]:
# primerjava po normah
zapis_podatkov(1,10, generiranje_tock_kvadrat, 50, 'n_od_1_do_10_primerjava_norme_1', None, graf, 1)
zapis_podatkov(1,10, generiranje_tock_kvadrat, 50, 'n_od_1_do_10_primerjava_norme_2', None, graf, 2)
zapis_podatkov(1,10, generiranje_tock_kvadrat, 50, 'n_od_1_do_10_primerjava_norme_Inf', None, graf, Infinity)

In [32]:
# natančnejši izračun pričakovane vrednosti (500 ponovitev) za n=3,4,5, kvadrat, poln graf, 2-norma
zapis_podatkov(3,5, generiranje_tock_kvadrat, 500, 'n_od_3_do_5_vsota')

In [0]:
# minimalne vsote in časovna zahtevnost najcenejšega prirejanja za večje n v navadnem polnem grafu in dvodelnem polnem grafu
zapis_podatkov(100, 100, generiranje_tock_kvadrat, 20, 'n_je_100_vsota', 'n_je_100_cas')
zapis_podatkov(1000, 1000, generiranje_tock_kvadrat, 20, 'n_je_1000_vsota', 'n_je_1000_cas')
zapis_podatkov(10000, 10000, generiranje_tock_kvadrat, 20, 'n_je_10000_vsota', 'n_je_10000_cas')

zapis_podatkov(100, 100, generiranje_tock_kvadrat, 20, 'n_je_100_vsota_dvodelni_graf', 'n_je_100_cas_dvodelni_graf', dvobarven_graf)
zapis_podatkov(1000, 1000, generiranje_tock_kvadrat, 20, 'n_je_1000_vsota_dvodelni_graf', 'n_je_1000_cas_dvodelni_graf', dvobarven_graf)
zapis_podatkov(10000, 10000, generiranje_tock_kvadrat, 20, 'n_je_10000_vsota_dvodelni_graf', 'n_je_10000_cas_dvodelni_graf', dvobarven_graf)